# Best location to sell organic products on marketplaces in Paris

**Important Note:**
<br>This notebook uses extensive parts of the code provided for educational purpose during the Coursera IBM Datascience professional certificate.

**Import of all the librairies needed**

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Data Preparation

In [2]:
# File present in the Github repository:
paris_marketplaces_data = pd.read_csv("liste_des_marches_de_quartier_a_paris.csv",delimiter=';')

In [3]:
paris_marketplaces_data.head()

,Type,Société gestionnaire,Marché,Arrondissement,Localisation,Adresse Complète (POI approchant),Lundi,Mardi,Mercredi,Jeudi,Vendredi,Samedi,Dimanche,Jour férié,geo_coordinates
0,Alimentaire découvert,Dadoun,AGUESSEAU,75008,"Place de la Madeleine, Bd Malherbes","Place de la Madeleine 75008 Paris, France",NaN,07h00 à 14h30,NaN,NaN,07h00 à 14h30,NaN,NaN,NaN,"48.8708, 2.325134"
1,Alimentaire découvert,Dadoun,AMIRAL BRUIX,75016,Bd de l'Amiral Bruix entre les rues Weber et M...,"Bd de l'Amiral Bruix 75016 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,"48.873657, 2.277084"
2,Alimentaire découvert,Dadoun,AUTEUIL,75016,Place Jean Lorrain,"Place Jean Lorrain 75016 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,"48.848042, 2.264235"
3,Alimentaire découvert,Dadoun,BARBES,75018,Bd de la Chapelle entre rues Guy Patin et Maub...,"Bd de la Chapelle 75018 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,"48.884376, 2.357139"
4,Alimentaire découvert,Dadoun,BERTHIER,75017,"Bd de Reims, le long du square A. Ulmann","Bd de Reims 75017 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,"48.88971, 2.29816"


In [4]:
paris_marketplaces_data.shape

(94, 15)

In [5]:
# Split the "geo_coordinates" column to 2 columns: "Latitude" & "Longitude"
paris_marketplaces_data[['geo_coordinates','Longitude']] = paris_marketplaces_data['geo_coordinates'].str.split(',',expand=True)

In [6]:
paris_marketplaces_data.head()

,Type,Société gestionnaire,Marché,Arrondissement,Localisation,Adresse Complète (POI approchant),Lundi,Mardi,Mercredi,Jeudi,Vendredi,Samedi,Dimanche,Jour férié,geo_coordinates,Longitude
0,Alimentaire découvert,Dadoun,AGUESSEAU,75008,"Place de la Madeleine, Bd Malherbes","Place de la Madeleine 75008 Paris, France",NaN,07h00 à 14h30,NaN,NaN,07h00 à 14h30,NaN,NaN,NaN,48.8708,2.325134
1,Alimentaire découvert,Dadoun,AMIRAL BRUIX,75016,Bd de l'Amiral Bruix entre les rues Weber et M...,"Bd de l'Amiral Bruix 75016 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,48.873657,2.277084
2,Alimentaire découvert,Dadoun,AUTEUIL,75016,Place Jean Lorrain,"Place Jean Lorrain 75016 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,48.848042,2.264235
3,Alimentaire découvert,Dadoun,BARBES,75018,Bd de la Chapelle entre rues Guy Patin et Maub...,"Bd de la Chapelle 75018 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,48.884376,2.357139
4,Alimentaire découvert,Dadoun,BERTHIER,75017,"Bd de Reims, le long du square A. Ulmann","Bd de Reims 75017 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,48.88971,2.29816


In [7]:
# Let's rename and translate in English the needed columns for the analysis of the dataset for a better understanding:
paris_marketplaces_data.rename(columns={'Société gestionnaire':'Administrator', 
                                        'Marché':'Marketplace',
                                        'Arrondissement':'City district',
                                        'Localisation':'Localization',
                                        'Adresse Complète (POI approchant)':'Address',
                                        'geo_coordinates':'Latitude'}, inplace=True)

In [8]:
paris_marketplaces_data.head()

,Type,Administrator,Marketplace,City district,Localization,Address,Lundi,Mardi,Mercredi,Jeudi,Vendredi,Samedi,Dimanche,Jour férié,Latitude,Longitude
0,Alimentaire découvert,Dadoun,AGUESSEAU,75008,"Place de la Madeleine, Bd Malherbes","Place de la Madeleine 75008 Paris, France",NaN,07h00 à 14h30,NaN,NaN,07h00 à 14h30,NaN,NaN,NaN,48.8708,2.325134
1,Alimentaire découvert,Dadoun,AMIRAL BRUIX,75016,Bd de l'Amiral Bruix entre les rues Weber et M...,"Bd de l'Amiral Bruix 75016 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,48.873657,2.277084
2,Alimentaire découvert,Dadoun,AUTEUIL,75016,Place Jean Lorrain,"Place Jean Lorrain 75016 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,48.848042,2.264235
3,Alimentaire découvert,Dadoun,BARBES,75018,Bd de la Chapelle entre rues Guy Patin et Maub...,"Bd de la Chapelle 75018 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,48.884376,2.357139
4,Alimentaire découvert,Dadoun,BERTHIER,75017,"Bd de Reims, le long du square A. Ulmann","Bd de Reims 75017 Paris, France",NaN,NaN,07h00 à 14h30,NaN,NaN,07h00 à 15h00,NaN,NaN,48.88971,2.29816


In [9]:
# Let's make sure all the city districts are from the inner center of Paris (starts with a 75):
paris_marketplaces_data['City district'].unique()

array([75008, 75016, 75018, 75017, 75002, 75011, 75014, 75015, 75012,
       75013, 75003, 75004, 75020, 75019, 75005, 75006, 75001, 75009,
       75010, 75007])

In [10]:
# Let's the various types of marketplaces:
paris_marketplaces_data['Type'].unique()

array(['Alimentaire découvert', 'Biologique', 'Alimentaire couvert',
       'Création', 'Fleurs', 'Puces', 'Oiseaux', 'Timbres'], dtype=object)

In [11]:
# Only keep the marketplaces related to food in our dataset:
foodtype = ['Alimentaire découvert', 'Biologique','Alimentaire couvert']
paris_marketplaces_data=paris_marketplaces_data.loc[paris_marketplaces_data['Type'].isin(foodtype)]

In [12]:
# Only keep the interesting colums for the analysis:
paris_marketplaces_data=paris_marketplaces_data[['Type','Administrator', 'Marketplace', 'City district',
                                                 'Localization',
                                                'Address', 'Latitude', 'Longitude']]

In [13]:
paris_marketplaces_data.head()

,Type,Administrator,Marketplace,City district,Localization,Address,Latitude,Longitude
0,Alimentaire découvert,Dadoun,AGUESSEAU,75008,"Place de la Madeleine, Bd Malherbes","Place de la Madeleine 75008 Paris, France",48.8708,2.325134
1,Alimentaire découvert,Dadoun,AMIRAL BRUIX,75016,Bd de l'Amiral Bruix entre les rues Weber et M...,"Bd de l'Amiral Bruix 75016 Paris, France",48.873657,2.277084
2,Alimentaire découvert,Dadoun,AUTEUIL,75016,Place Jean Lorrain,"Place Jean Lorrain 75016 Paris, France",48.848042,2.264235
3,Alimentaire découvert,Dadoun,BARBES,75018,Bd de la Chapelle entre rues Guy Patin et Maub...,"Bd de la Chapelle 75018 Paris, France",48.884376,2.357139
4,Alimentaire découvert,Dadoun,BERTHIER,75017,"Bd de Reims, le long du square A. Ulmann","Bd de Reims 75017 Paris, France",48.88971,2.29816


In [14]:
paris_marketplaces_data.shape

(83, 8)

In [15]:
# Check the data types of the columns:
paris_marketplaces_data.dtypes

Type             object
Administrator    object
Marketplace      object
City district     int64
Localization     object
Address          object
Latitude         object
Longitude        object
dtype: object

In [16]:
# Convert the columns to integer/float when needed:
paris_marketplaces_data['Latitude']=paris_marketplaces_data['Latitude'].astype(float)
paris_marketplaces_data['Longitude']=paris_marketplaces_data['Longitude'].astype(float)

In [17]:
paris_marketplaces_data.dtypes

Type              object
Administrator     object
Marketplace       object
City district      int64
Localization      object
Address           object
Latitude         float64
Longitude        float64
dtype: object

## Geographical representation of the data (marketplaces)

In [18]:
# Use geopy library to get the latitude and longitude values of Paris.

# In order to define an instance of the geocoder, we need to define a user_agent.
# We will name our agent ny_explorer, as shown below:
address = 'Paris, France'

geolocator = Nominatim(user_agent="paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris City are 48.8566101, 2.3514992.


In [19]:
# Create a map of Paris with metrostation superimposed on top.

# Create map of Paris using latitude and longitude values:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map:
for lat, lng, marketplace in zip(paris_marketplaces_data['Latitude'], paris_marketplaces_data['Longitude'], paris_marketplaces_data['Marketplace']):
    label = '{}'.format(marketplace)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

## Foursquare Credentials and Version

In [20]:
# Define Foursquare Credentials and Version:

CLIENT_ID = '1C1XLXPREYJOYNTXSAKBBDAS2SVZSRRZOLWDS54KKUDZOPYZ' # your Foursquare ID
CLIENT_SECRET = 'TALOOIKW4G5KCX5KUYRRTZW5OWEQ4XCZ35GQLHX20JZSEWPL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1C1XLXPREYJOYNTXSAKBBDAS2SVZSRRZOLWDS54KKUDZOPYZ
CLIENT_SECRET:TALOOIKW4G5KCX5KUYRRTZW5OWEQ4XCZ35GQLHX20JZSEWPL


## Data exploration

In [21]:
LIMIT = 10 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

In [22]:
# Let's create a function to explore all of the marketplaces in Paris:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Create the API request URL:
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Make the GET request:
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Return only relevant information for each nearby venue:
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Marketplace', 
                  'Marketplace Latitude', 
                  'Marketplace Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# Now we write the code to run the above function on each marketplace and create a new dataframe
# called paris_venues:
paris_venues = getNearbyVenues(names=paris_marketplaces_data['Marketplace'],
                                   latitudes=paris_marketplaces_data['Latitude'],
                                   longitudes=paris_marketplaces_data['Longitude']
                                  )

AGUESSEAU
AMIRAL BRUIX
AUTEUIL
BARBES
BERTHIER
BOURSE
CHARONNE
EDGAR QUINET
GRENELLE
LECOURBE
LEDRU ROLLIN
MAISON BLANCHE
POINT DU JOUR
PORTE MOLITOR
Saint Didier
BRANCUSI
Enfants rouges
Saint Didier
La Chapelle
AUGUSTE BLANQUI
BAUDOYER
BELGRAND
BOBILLOT
COURS DE VINCENNES
CRIMEE CURIAL
DAUMESNIL
DAVOUT
LEFEBVRE
MAUBERT
MONGE
NAVIER
PARIS RIVE GAUCHE
POPINCOURT
RASPAIL
SAINT CHARLES
SAINT HONORE
SALPETRIERE
Saint Germain
ANVERS
BASTILLE
BELLEVILLE
CERVANTES
GROS LA FONTAINE
ORDENER
PERE LACHAISE
PORTE DOREE
VINCENT AURIOL
BEAUVAU
Saint Quentin
Beauvau
Ternes
ALESIA
ALIBERT
CONVENTION
JEAN JAURES
JOINVILLE
MORTIER
MOUTON DUVERNET
NEY
ORNANO
PRESIDENT WILSON
PYRENEES
REUNION
SAINT ELOI
SAINT EUSTACHE
TELEGRAPHE
VILLEMAIN
RASPAIL
Treilhard
Saint Martin
Passy
BERCY
BRASSENS
BRUNE
GEORGES BRASSENS
JEANNE D'ARC
PLACE DES FETES
PORT ROYAL
PORTE BRUNET
SAXE BRETEUIL
VILLETTE
BATIGNOLES
Batignolles


In [24]:
# Let's check the size of the resulting dataframe:
print(paris_venues.shape)
paris_venues.head()

(829, 7)


,Marketplace,Marketplace Latitude,Marketplace Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AGUESSEAU,48.8708,2.325134,Café Pouchkine,48.869627,2.325228,Dessert Shop
1,AGUESSEAU,48.8708,2.325134,Patrick Roger,48.869512,2.323673,Dessert Shop
2,AGUESSEAU,48.8708,2.325134,Place de la Madeleine,48.869606,2.324807,Plaza
3,AGUESSEAU,48.8708,2.325134,La Maison de la Truffe,48.870747,2.324374,French Restaurant
4,AGUESSEAU,48.8708,2.325134,Fauchon – L'Épicerie,48.870694,2.325414,Gourmet Shop


In [25]:
# Let's check how many venues were returned for each Marketplace:
paris_venues.groupby('Marketplace').count()

,Marketplace Latitude,Marketplace Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Marketplace,,,,,,
AGUESSEAU,10,10,10,10,10,10
ALESIA,10,10,10,10,10,10
ALIBERT,10,10,10,10,10,10
AMIRAL BRUIX,10,10,10,10,10,10
ANVERS,10,10,10,10,10,10
AUGUSTE BLANQUI,10,10,10,10,10,10
AUTEUIL,10,10,10,10,10,10
BARBES,10,10,10,10,10,10
BASTILLE,10,10,10,10,10,10


In [26]:
# Let's find out how many unique categories can be curated from all the returned venues:
print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 147 uniques categories.


## Data (Marketplaces) analysis

In [27]:
# Analyze Each marketplace.
# One hot encoding:
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# Add marketplace column back to dataframe:
paris_onehot['Marketplace'] = paris_venues['Marketplace'] 

# Move marketplace column to the first column:
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Marketplace,African Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Burger Joint,Butcher,Café,Cambodian Restaurant,Camera Store,Canal,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Flea Market,Food & Drink Shop,Food Truck,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lebanese Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Moroccan Restaurant,Motel,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Radio Station,Restaurant,Rock Club,Russian Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Plaza,Soccer Field,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,AGUESSEAU,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,AGUESSEAU,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,AGUESSEAU,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,AGUESSEAU,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,AGUESSEAU,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
paris_onehot.shape

(829, 148)

In [29]:
# Next, let's group rows by marketplace and by taking the mean of the frequency of occurrence of each category:
paris_grouped = paris_onehot.groupby('Marketplace').mean().reset_index()
paris_grouped.head()

,Marketplace,African Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Burger Joint,Butcher,Café,Cambodian Restaurant,Camera Store,Canal,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Flea Market,Food & Drink Shop,Food Truck,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lebanese Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Moroccan Restaurant,Motel,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Radio Station,Restaurant,Rock Club,Russian Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Plaza,Soccer Field,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,AGUESSEAU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ALESIA,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ALIBERT,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AMIRAL BRUIX,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [30]:
# Let's confirm the new size:
paris_grouped.shape

(81, 148)

In [31]:
# Let's print each marketplace along with the top 5 most common venues:
num_top_venues = 5

for hood in paris_grouped['Marketplace']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Marketplace'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AGUESSEAU----
                venue  freq
0        Gourmet Shop   0.3
1        Dessert Shop   0.2
2  Russian Restaurant   0.1
3               Plaza   0.1
4            Boutique   0.1


----ALESIA----
                  venue  freq
0     French Restaurant   0.2
1                Bakery   0.1
2          Burger Joint   0.1
3  Ethiopian Restaurant   0.1
4                  Park   0.1


----ALIBERT----
                  venue  freq
0  Cambodian Restaurant   0.2
1           Coffee Shop   0.1
2                   Bar   0.1
3             Gastropub   0.1
4            Restaurant   0.1


----AMIRAL BRUIX----
               venue  freq
0  French Restaurant   0.2
1          Hotel Bar   0.1
2      Train Station   0.1
3        Pizza Place   0.1
4              Plaza   0.1


----ANVERS----
               venue  freq
0             Bakery   0.3
1       Concert Hall   0.1
2        Pizza Place   0.1
3        Music Venue   0.1
4  French Restaurant   0.1


----AUGUSTE BLANQUI----
                      venue  

                 venue  freq
0    French Restaurant   0.2
1  Lebanese Restaurant   0.1
2          Cheese Shop   0.1
3          Pastry Shop   0.1
4               Bakery   0.1


----MONGE----
              venue  freq
0          Creperie   0.2
1       Coffee Shop   0.1
2               Bar   0.1
3     Historic Site   0.1
4  Greek Restaurant   0.1


----MORTIER----
                 venue  freq
0                Plaza   0.2
1          Pizza Place   0.2
2               Bakery   0.2
3  Japanese Restaurant   0.1
4             Pharmacy   0.1


----MOUTON DUVERNET----
                   venue  freq
0      French Restaurant   0.6
1         Ice Cream Shop   0.1
2  Vietnamese Restaurant   0.1
3               Tea Room   0.1
4              Brasserie   0.1


----NAVIER----
               venue  freq
0         Restaurant   0.2
1  French Restaurant   0.2
2          Wine Shop   0.1
3    Cultural Center   0.1
4             Bakery   0.1


----NEY----
                      venue  freq
0        African Restau

In [32]:
# Let's put that into a pandas dataframe:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues:
columns = ['Marketplace']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe:
Marketplace_venues_sorted = pd.DataFrame(columns=columns)
Marketplace_venues_sorted['Marketplace'] = paris_grouped['Marketplace']

for ind in np.arange(paris_grouped.shape[0]):
    Marketplace_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

Marketplace_venues_sorted.head()

,Marketplace,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AGUESSEAU,Gourmet Shop,Dessert Shop,Plaza,Russian Restaurant,Chocolate Shop,French Restaurant,Boutique,Gift Shop,Dim Sum Restaurant,Cocktail Bar
1,ALESIA,French Restaurant,Burger Joint,Supermarket,Arts & Crafts Store,Park,Italian Restaurant,Japanese Restaurant,Bakery,Ethiopian Restaurant,Women's Store
2,ALIBERT,Cambodian Restaurant,Bar,Coffee Shop,Cheese Shop,Italian Restaurant,Asian Restaurant,Restaurant,Gastropub,Canal,Cultural Center
3,AMIRAL BRUIX,French Restaurant,Pizza Place,Hotel,Art Museum,Hotel Bar,Train Station,Plaza,Supermarket,Bed & Breakfast,Women's Store
4,ANVERS,Bakery,Concert Hall,Cocktail Bar,Music Venue,French Restaurant,Pizza Place,Hotel,Wine Shop,Comic Shop,Comfort Food Restaurant


## Cluster of the marketplaces in Paris (5 clusters)

In [34]:
# Cluster of the marketplaces in Paris.
# Run k-means to cluster the marketplaces into 5 clusters.
# Set number of clusters:
kclusters = 5

paris_grouped_clustering = paris_grouped.drop('Marketplace', 1)

# Run k-means clustering:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# Check cluster labels generated for each row in the dataframe:
kmeans.labels_[0:10] 

array([4, 2, 0, 2, 1, 2, 4, 0, 4, 2], dtype=int32)

In [35]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each marketplace.
# Add clustering labels:
Marketplace_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = paris_marketplaces_data

# Merge DowntownToronto_grouped with DowntownToronto_data to add latitude/longitude for each marketplace:
paris_merged = paris_merged.join(Marketplace_venues_sorted.set_index('Marketplace'), on='Marketplace')

paris_merged.head()

,Type,Administrator,Marketplace,City district,Localization,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alimentaire découvert,Dadoun,AGUESSEAU,75008,"Place de la Madeleine, Bd Malherbes","Place de la Madeleine 75008 Paris, France",48.870800,2.325134,4,Gourmet Shop,Dessert Shop,Plaza,Russian Restaurant,Chocolate Shop,French Restaurant,Boutique,Gift Shop,Dim Sum Restaurant,Cocktail Bar
1,Alimentaire découvert,Dadoun,AMIRAL BRUIX,75016,Bd de l'Amiral Bruix entre les rues Weber et M...,"Bd de l'Amiral Bruix 75016 Paris, France",48.873657,2.277084,2,French Restaurant,Pizza Place,Hotel,Art Museum,Hotel Bar,Train Station,Plaza,Supermarket,Bed & Breakfast,Women's Store
2,Alimentaire découvert,Dadoun,AUTEUIL,75016,Place Jean Lorrain,"Place Jean Lorrain 75016 Paris, France",48.848042,2.264235,4,Brasserie,Italian Restaurant,Historic Site,Dessert Shop,Burger Joint,Middle Eastern Restaurant,French Restaurant,Market,Pizza Place,Asian Restaurant
3,Alimentaire découvert,Dadoun,BARBES,75018,Bd de la Chapelle entre rues Guy Patin et Maub...,"Bd de la Chapelle 75018 Paris, France",48.884376,2.357139,0,Indian Restaurant,Bistro,Concert Hall,Supermarket,Restaurant,Theater,Café,Women's Store,Deli / Bodega,Diner
4,Alimentaire découvert,Dadoun,BERTHIER,75017,"Bd de Reims, le long du square A. Ulmann","Bd de Reims 75017 Paris, France",48.889710,2.298160,1,Clothing Store,Hotel,Toy / Game Store,Moroccan Restaurant,Multiplex,Sandwich Place,Bagel Shop,Bakery,Creperie,Cosmetics Shop


In [36]:
# Finally, let's visualize the resulting clusters.
# Create map:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set color scheme for the clusters:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map:
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Marketplace'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Clusters examination

In [37]:
### Cluster 1

In [38]:
paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[2] + list(range(5, paris_merged.shape[1]))]]

,Marketplace,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,BARBES,"Bd de la Chapelle 75018 Paris, France",48.884376,2.357139,0,Indian Restaurant,Bistro,Concert Hall,Supermarket,Restaurant,Theater,Café,Women's Store,Deli / Bodega,Diner
26,CRIMEE CURIAL,"Rue de Crimée 75019 Paris, France",48.890575,2.376821,0,Café,Asian Restaurant,Hostel,Brewery,French Restaurant,Beer Store,Beer Bar,Bed & Breakfast,Ethiopian Restaurant,Electronics Store
39,Saint Germain,"4/8, Rue Lobineau 75006 Paris, France",48.851711,2.336460,0,Wine Bar,Miscellaneous Shop,Tea Room,Breton Restaurant,Electronics Store,Bistro,Ice Cream Shop,Seafood Restaurant,Beer Bar,Diner
42,BELLEVILLE,"Bd de Belleville 75011 Paris, France",48.869495,2.379707,0,Bar,Pizza Place,Restaurant,Spanish Restaurant,Hostel,Coffee Shop,Cocktail Bar,Comic Shop,Comfort Food Restaurant,Concert Hall
45,ORDENER,"rue Montcalm 75018 Paris, France",48.893135,2.337854,0,Sports Club,Wine Bar,Arepa Restaurant,Pizza Place,Dessert Shop,Bistro,Burger Joint,Restaurant,Bar,French Restaurant
46,PERE LACHAISE,"Bd de Ménilmontant 75011 Paris, France",48.862064,2.387661,0,Bar,Greek Restaurant,Bistro,French Restaurant,Restaurant,Cemetery,Moroccan Restaurant,Art Museum,Wine Shop,Corsican Restaurant
48,VINCENT AURIOL,"64 Bd Vincent Auriol 75013 Paris, France",48.834549,2.366460,0,Taco Place,Chinese Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Brazilian Restaurant,Park,Pub,Nightclub,Supermarket,Sushi Restaurant
55,ALIBERT,"rue Alibert 75010 Paris, France",48.871651,2.367655,0,Cambodian Restaurant,Bar,Coffee Shop,Cheese Shop,Italian Restaurant,Asian Restaurant,Restaurant,Gastropub,Canal,Cultural Center
58,JOINVILLE,"Place de Joinville 75019 Paris, France",48.889530,2.380587,0,Beer Bar,Hostel,Restaurant,Canal,Café,Brewery,Coffee Shop,Bistro,Pizza Place,Garden
61,NEY,"Bd Ney 75018 Paris, France",48.898109,2.346444,0,African Restaurant,Italian Restaurant,Flea Market,Mediterranean Restaurant,Beer Garden,Deli / Bodega,Bar,Garden,Vietnamese Restaurant,Gym / Fitness Center


In [39]:
### Cluster 2

In [40]:
paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[2] + list(range(5, paris_merged.shape[1]))]]

,Marketplace,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,BERTHIER,"Bd de Reims 75017 Paris, France",48.889710,2.298160,1,Clothing Store,Hotel,Toy / Game Store,Moroccan Restaurant,Multiplex,Sandwich Place,Bagel Shop,Bakery,Creperie,Cosmetics Shop
8,GRENELLE,"Bd de Grenelle 75015 Paris, France",48.849007,2.297894,1,Hotel,French Restaurant,Bakery,Corsican Restaurant,Creperie,Market,Korean Restaurant,Department Store,Electronics Store,Diner
10,LEDRU ROLLIN,"Av. Ledru Rollin 75012 Paris, France",48.848144,2.371697,1,Cocktail Bar,Trail,Bridge,French Restaurant,Bar,Creperie,Camera Store,Pedestrian Plaza,Hotel,Corsican Restaurant
22,BAUDOYER,"Place Baudoyer 75004 Paris, France",48.856209,2.355425,1,Coffee Shop,Tea Room,Bookstore,Art Gallery,Portuguese Restaurant,Ice Cream Shop,Dessert Shop,Pastry Shop,French Restaurant,Café
25,COURS DE VINCENNES,"Bd Picpus 75012 Paris, France",48.839699,2.400260,1,Plaza,Bakery,Farmers Market,Food & Drink Shop,Bookstore,Bistro,Bar,French Restaurant,Ice Cream Shop,Hotel
28,DAVOUT,"Bd Davout 75020 Paris, France",48.856007,2.410118,1,Hotel,Plaza,Stadium,Cosmetics Shop,Bakery,Brazilian Restaurant,Sandwich Place,Music Venue,French Restaurant,Women's Store
30,MAUBERT,"Place Maubert 75005 Paris, France",48.849846,2.348470,1,French Restaurant,Comic Shop,Hotel,Cheese Shop,Coffee Shop,Japanese Restaurant,Pastry Shop,Bakery,Lebanese Restaurant,Women's Store
31,MONGE,"Place Monge 75005 Paris, France",48.843224,2.351440,1,Creperie,Plaza,Bar,Ice Cream Shop,Greek Restaurant,French Restaurant,Peruvian Restaurant,Coffee Shop,Historic Site,Bagel Shop
33,PARIS RIVE GAUCHE,"Rue Jean Anouilh 75013 Paris, France",48.831783,2.377777,1,French Restaurant,Lounge,Performing Arts Venue,Plaza,Bookstore,Steakhouse,Multiplex,Rock Club,Café,Food Truck
36,SAINT CHARLES,"rue Saint Charles 75015 Paris, France",48.844807,2.283913,1,Flea Market,Gourmet Shop,Dessert Shop,Department Store,Park,Korean Restaurant,Bakery,Cheese Shop,Hotel,Gym / Fitness Center


In [41]:
### Cluster 3

In [42]:
paris_merged.loc[paris_merged['Cluster Labels'] == 2, paris_merged.columns[[2] + list(range(5, paris_merged.shape[1]))]]

,Marketplace,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,AMIRAL BRUIX,"Bd de l'Amiral Bruix 75016 Paris, France",48.873657,2.277084,2,French Restaurant,Pizza Place,Hotel,Art Museum,Hotel Bar,Train Station,Plaza,Supermarket,Bed & Breakfast,Women's Store
7,EDGAR QUINET,"Bd Edgar Quinet 75014 Paris, France",48.841908,2.322674,2,French Restaurant,Japanese Restaurant,Creperie,Scenic Lookout,Comfort Food Restaurant,Argentinian Restaurant,Ice Cream Shop,Auvergne Restaurant,Electronics Store,Diner
11,MAISON BLANCHE,"Av. d'Italie 75013 Paris, France",48.825085,2.357587,2,Vietnamese Restaurant,French Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Japanese Restaurant,Bistro,Gourmet Shop,Greek Restaurant,Comfort Food Restaurant
12,POINT DU JOUR,"Av. de Versailles 75016 Paris, France",48.842896,2.269147,2,Park,Italian Restaurant,Beer Garden,French Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Corsican Restaurant,Concert Hall,Creperie
15,BRANCUSI,"Place Constantin Brancusi 75014 Paris, France",48.836990,2.320652,2,French Restaurant,Motel,Cambodian Restaurant,Hookah Bar,Hotel,Art Museum,Coffee Shop,Japanese Restaurant,Seafood Restaurant,Diner
16,Enfants rouges,"33, Rue de Bretagne 75003 Paris, France",48.862759,2.362790,2,Sandwich Place,Cocktail Bar,Italian Restaurant,Farmers Market,Burger Joint,Japanese Restaurant,Speakeasy,Corsican Restaurant,Concert Hall,Cosmetics Shop
18,La Chapelle,"10, Rue de l'Olive 75018 Paris, France",48.890949,2.361250,2,French Restaurant,Coffee Shop,Asian Restaurant,Farmers Market,Café,Garden,Theater,Bagel Shop,Wine Bar,Cosmetics Shop
21,AUGUSTE BLANQUI,"Bd Auguste Blanqui 75013 Paris, France",48.829708,2.348003,2,Thai Restaurant,Burger Joint,Park,Mediterranean Restaurant,Italian Restaurant,French Restaurant,Café,Bakery,Bar,Basque Restaurant
24,BOBILLOT,"place de Rungis 75013 Paris, France",48.822304,2.346452,2,Chinese Restaurant,Café,Pub,Beer Store,French Restaurant,Bakery,Cheese Shop,Vietnamese Restaurant,Concert Hall,Corsican Restaurant
32,NAVIER,"Rue Navier 75017 Paris, France",48.895794,2.325054,2,Restaurant,French Restaurant,Bar,Wine Shop,Pedestrian Plaza,Bistro,Bakery,Cultural Center,Department Store,Electronics Store


In [43]:
### Cluster 4

In [44]:
paris_merged.loc[paris_merged['Cluster Labels'] == 3, paris_merged.columns[[2] + list(range(5, paris_merged.shape[1]))]]

,Marketplace,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,BOURSE,"Place de la Bourse 75002 Paris, France",48.868584,2.341653,3,French Restaurant,Pizza Place,Nightclub,Wine Bar,Historic Site,Cocktail Bar,Italian Restaurant,Tapas Restaurant,Dim Sum Restaurant,Dessert Shop
6,CHARONNE,"Bd de Charonne 75011 Paris, France",48.854843,2.395497,3,French Restaurant,Bar,Bookstore,Vegetarian / Vegan Restaurant,Creperie,Bagel Shop,Tech Startup,Women's Store,Electronics Store,Diner
9,LECOURBE,"Rue Lecourbe 75015 Paris, France",48.841740,2.299723,3,French Restaurant,Hotel,Indian Restaurant,Restaurant,Japanese Restaurant,Bakery,Park,Women's Store,Cultural Center,Diner
14,Saint Didier,"rue Mesnil 75016 Paris, France",48.867481,2.283890,3,French Restaurant,Hotel,Italian Restaurant,Bakery,Pizza Place,Burger Joint,Deli / Bodega,Diner,Dim Sum Restaurant,Dessert Shop
17,Saint Didier,"rue saint didier 75016 Paris, France",48.866558,2.285194,3,French Restaurant,Hotel,Italian Restaurant,Bakery,Pizza Place,Burger Joint,Deli / Bodega,Diner,Dim Sum Restaurant,Dessert Shop
23,BELGRAND,"place Edith Piaf 75020 Paris, France",48.864788,2.404750,3,French Restaurant,Plaza,Italian Restaurant,Park,Bookstore,Bistro,Mexican Restaurant,Gift Shop,Electronics Store,Comfort Food Restaurant
29,LEFEBVRE,"Bd Lefebvre 75015 Paris, France",48.830219,2.294734,3,French Restaurant,Peruvian Restaurant,Park,Moroccan Restaurant,Theater,Vietnamese Restaurant,Chinese Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall
35,RASPAIL,"Bd Raspail 75006 Paris, France",48.842701,2.329470,3,French Restaurant,Cocktail Bar,Japanese Restaurant,Steakhouse,Seafood Restaurant,Theater,Noodle House,Hotel,Corsican Restaurant,Cosmetics Shop
43,CERVANTES,"Rue Barque 75015 Paris, France",48.838577,2.307954,3,French Restaurant,Plaza,Bar,Gastropub,Japanese Restaurant,Korean Restaurant,Gym / Fitness Center,Vietnamese Restaurant,Deli / Bodega,Ethiopian Restaurant
60,MOUTON DUVERNET,"Place Jacques Demy 75014 Paris, France",48.831882,2.327286,3,French Restaurant,Vietnamese Restaurant,Ice Cream Shop,Brasserie,Tea Room,Women's Store,Deli / Bodega,Electronics Store,Diner,Dim Sum Restaurant


In [45]:
### Cluster 5

In [46]:
paris_merged.loc[paris_merged['Cluster Labels'] == 4, paris_merged.columns[[2] + list(range(5, paris_merged.shape[1]))]]

,Marketplace,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AGUESSEAU,"Place de la Madeleine 75008 Paris, France",48.870800,2.325134,4,Gourmet Shop,Dessert Shop,Plaza,Russian Restaurant,Chocolate Shop,French Restaurant,Boutique,Gift Shop,Dim Sum Restaurant,Cocktail Bar
2,AUTEUIL,"Place Jean Lorrain 75016 Paris, France",48.848042,2.264235,4,Brasserie,Italian Restaurant,Historic Site,Dessert Shop,Burger Joint,Middle Eastern Restaurant,French Restaurant,Market,Pizza Place,Asian Restaurant
13,PORTE MOLITOR,"porte de Molitor 75016 Paris, France",48.845253,2.261390,4,French Restaurant,Gym / Fitness Center,Market,Chinese Restaurant,Dessert Shop,Brasserie,Middle Eastern Restaurant,Pharmacy,Italian Restaurant,Gastropub
27,DAUMESNIL,"Rue de Reuilly 75012 Paris, France",48.844673,2.389714,4,Chinese Restaurant,Japanese Restaurant,Creperie,Furniture / Home Store,Garden,Bakery,Thai Restaurant,Bistro,Turkish Restaurant,Gift Shop
34,POPINCOURT,"Bd Richard Lenoir 75011 Paris, France",48.860321,2.372673,4,Coffee Shop,French Restaurant,Vegetarian / Vegan Restaurant,Playground,Bed & Breakfast,Falafel Restaurant,Chinese Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall
41,BASTILLE,"Bd Richard Lenoir 75011 Paris, France",48.860321,2.372673,4,Coffee Shop,French Restaurant,Vegetarian / Vegan Restaurant,Playground,Bed & Breakfast,Falafel Restaurant,Chinese Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall
63,PRESIDENT WILSON,"Av. du Président Wilson 75016 Paris, France",48.864792,2.295730,4,Art Museum,French Restaurant,Hotel,Hotel Bar,Bookstore,Museum,Market,Women's Store,Dessert Shop,Electronics Store
73,Passy,"1, Rue Bois le Vent 75016 Paris, France",48.857590,2.277280,4,French Restaurant,Middle Eastern Restaurant,Dessert Shop,Gourmet Shop,Restaurant,Food & Drink Shop,Italian Restaurant,Japanese Restaurant,Market,Deli / Bodega
